In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#host = 'https://api.the-odds-api.com'
#key = '4ef1ba01e4f76c06985b79b89b3d37f7'
#link = host+'/v4/sports/americanfootball_nfl/odds?regions=us&oddsFormat=american&apiKey='+key
#r = requests.get(link)
#data = r.json()
#r= requests.get(r"https://www.footballdb.com/scores/index.html?lg=NFL&yr=2022")

In [3]:
data = pd.read_csv('C:/Users/JackR/Documents/Business/Betting/Data/nfl_2012_2022_week17.csv')

In [4]:
data.replace('Washington Redskins','Washington Commanders', inplace=True)
data.replace('Washington Football Team','Washington Commanders', inplace=True)
data.replace('Oakland Raiders','Las Vegas Raiders', inplace=True)
data.replace('St. Louis Rams','Los Angeles Rams', inplace=True)
data.replace('San Diego Chargers','Los Angeles Chargers', inplace=True)

In [5]:
teams = dict(zip(data.Home.unique(),np.array(list(range(len(data.Home.unique()))))))

In [6]:
teams_list = list(zip(data.Home.unique(),np.array(list(range(len(data.Home.unique()))))))

In [7]:
for i in range(len(data)):
    data['Visitor'][i] = teams[data['Visitor'][i]]
    data['Home'][i] = teams[data['Home'][i]]

C:\Users\JackR\AppData\Local\Temp\ipykernel_3152\185245688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Visitor'][i] = teams[data['Visitor'][i]]
C:\Users\JackR\AppData\Local\Temp\ipykernel_3152\185245688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Home'][i] = teams[data['Home'][i]]


# Colley Matrix Model

In [8]:
team_name = [teams_list[i][0] for i in range(len(teams_list))]
rankings = pd.DataFrame({"team":list(teams.values()),"name":team_name})

In [9]:
year = 2022
colley = data.loc[(data['Week']!='Postseason') & (data['Year']==year)]

In [10]:
colley.reset_index(drop=True, inplace=True)

In [11]:
n = colley.groupby('Home').count()['Year'] + colley.groupby('Visitor').count()['Year']

In [12]:
C = np.zeros((len(teams),len(teams)))

In [13]:
np.fill_diagonal(C, 2 + n)

In [14]:
for i in range(len(teams)):
    a = np.array(colley['Home'][np.where(colley['Visitor']==i)[0]])
    b = np.array(colley['Visitor'][np.where(colley['Home']==i)[0]])
    c = np.concatenate((a,b))
    t, ntimes = np.unique(np.sort(c), return_counts=True)
    for pos, j in enumerate(t):
        if i == int(j):
            continue
        else:
            C[i,j] = ntimes[pos]

In [15]:
visitor_wins = colley['Visitor'][(colley['AwayScore'] > colley['HomeScore'])]
home_wins = colley['Home'][(colley['AwayScore'] < colley['HomeScore'])]

visitor_wins = np.unique(np.sort(visitor_wins), return_counts=True)
home_wins = np.unique(np.sort(home_wins), return_counts=True)


#home_wins = np.insert(home_wins[0],4,4),np.insert(home_wins[1],4,0)
#visitor_wins = np.insert(visitor_wins[0],2,2),np.insert(visitor_wins[1],2,0)
#visitor_wins = np.insert(visitor_wins[0],22,22),np.insert(visitor_wins[1],22,0)

In [16]:
wins = visitor_wins[1] + home_wins[1]

ValueError: operands could not be broadcast together with shapes (32,) (31,) 

In [17]:
visitor_losses = colley['Visitor'][(colley['AwayScore'] <= colley['HomeScore'])]
home_losses = colley['Home'][(colley['AwayScore'] >= colley['HomeScore'])]

visitor_losses = np.unique(np.sort(visitor_losses), return_counts=True)
home_losses = np.unique(np.sort(home_losses), return_counts=True)

#visitor_losses = np.insert(visitor_losses[0],6,6),np.insert(visitor_losses[1],6,0)
#home_losses = np.insert(home_losses[0],12,12),np.insert(home_losses[1],12,0)
#home_losses = np.insert(home_losses[0],20,20),np.insert(home_losses[1],20,0)

#home_losses = (colley['AwayScore'] >= colley['HomeScore'])
#colley[home_losses].groupby('Home').count()['Year'].to_numpy()

In [18]:
losses = visitor_losses[1] + home_losses[1]

In [19]:
b = 1 + (wins - losses)/2

NameError: name 'wins' is not defined

In [1007]:
r = np.linalg.inv(C).dot(b)

In [1008]:
rankings[str(year)] = r

In [1010]:
rankings.to_csv('C:/Users/JackR/Documents/Business/Betting/Data/nfl_rankings_2012_2022.csv')

In [616]:
#rankings.sort_values(by='rank',inplace=True, ascending=False)

In [634]:
#data.loc[(data['Week']=='Postseason') & (data['Year']==2012)]

In [635]:
#playoff_teams = ['Kansas City Chiefs','Buffalo Bills','Cincinnati Bengals',
#                 'Jacksonville Jaguars','Los Angeles Chargers','Baltimore Ravens',
#                 'Miami Dolphins','Philadelphia Eagles','San Francisco 49ers',
#                 'Minnesota Vikings','Tampa Bay Buccaneers','Dallas Cowboys',
#                 'New York Giants','Seattle Seahawks']

#rankings[rankings['name'].isin(playoff_teams)]

# Dynamic Bi-Variate Poisson Model

In [490]:
# to obtain the strength of each team's offense and defense we can use LSTM to estimate it over time
# also can use a normally distributed sample of a teams win/lose margin to estimate lambda_x and lambda_y

In [1126]:
dynamic_poisson = data

In [1127]:
dynamic_poisson['diff'] = dynamic_poisson['AwayScore'] - dynamic_poisson['HomeScore']

In [1128]:
dynamic_poisson = dynamic_poisson[(dynamic_poisson['Week']=='Postseason') & (dynamic_poisson['Year']==2021)]

In [1140]:
rankings.max(level=0)

C:\Users\JackR\AppData\Local\Temp\ipykernel_23576\123203926.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  rankings.max(level=0)


,team,name,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,New York Giants,0.065911,-0.051576,-0.096982,-0.085936,0.217582,-0.275094,-0.164858,-0.177182,-0.086165,-0.241893,0.035086
1,1,New Orleans Saints,-0.048672,0.190612,-0.022639,-0.036705,-0.046183,0.182867,0.327453,0.327779,0.296772,0.036634,-0.064921
2,2,Detroit Lions,-0.244603,0.012102,0.222711,-0.045430,0.109695,0.096805,-0.086609,-0.258469,-0.151951,-0.288508,0.030157
3,3,Tennessee Titans,-0.094666,-0.029026,-0.317511,-0.247602,0.118071,0.134761,0.064747,0.096518,0.219973,0.238369,-0.054431
4,4,Houston Texans,0.254540,-0.355414,0.125445,0.077795,0.084341,-0.221886,0.225462,0.122312,-0.233400,-0.210069,-0.241976
5,5,New York Jets,-0.092062,0.044325,-0.222893,0.187294,-0.158839,-0.149837,-0.196413,-0.012237,-0.387046,-0.231936,-0.076148
6,6,Kansas City Chiefs,-0.333529,0.243482,0.091653,0.207057,0.252247,0.158943,0.275933,0.240801,0.403815,0.178412,0.351806
7,7,Minnesota Vikings,0.134365,-0.159029,-0.003480,0.202585,0.037275,0.323988,0.021030,0.161174,-0.038360,0.001620,0.305762
8,8,Cleveland Browns,-0.156446,-0.214789,-0.014622,-0.275706,-0.397926,-0.443612,-0.036567,-0.105305,0.246201,-0.004434,-0.062800
9,9,Chicago Bears,0.144286,0.058233,-0.166674,-0.115371,-0.267289,-0.183949,0.316169,0.018813,0.036045,-0.118717,-0.318767


In [1130]:
dynamic_poisson

,Year,Week,Visitor,AwayScore,Home,HomeScore,diff
2677,2021,Postseason,15,19,16,26,-7
2678,2021,Postseason,20,17,17,47,-30
2679,2021,Postseason,18,15,10,31,-16
2680,2021,Postseason,28,23,31,17,6
2681,2021,Postseason,27,21,6,42,-21
2682,2021,Postseason,11,11,24,34,-23
2683,2021,Postseason,16,19,3,16,3
2684,2021,Postseason,28,13,12,10,3
2685,2021,Postseason,24,30,10,27,3
2686,2021,Postseason,17,36,6,42,-6


In [25]:
pd.read_csv("C:/Documents\Business\Betting\Data\nfl_rankings_2012_2022.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Documents\\Business\\Betting\\Data\nfl_rankings_2012_2022.csv'